In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from state import get_state, queue
import os
import traci

In [ ]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG")

In [40]:
import tensorflow as tf
import numpy as np
from collections import deque

In [41]:
# from tensorflow.keras import Sequential, layers, models, activations
# from tensorflow.random import set_seed
# import numpy as np
# from collections import deque
# from tensorflow import keras
# from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
# from tensorflow.keras.losses import MeanSquaredError
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from tensorflow.keras.layers import Dense, ReLU


In [42]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = tf.keras.Sequential([
    tf.keras.layers.Dense(32,activation='relu', input_shape=input_shape),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(n_outputs, activation= 'linear')])

In [43]:
target = tf.keras.models.clone_model(model_action)  # clone the model's architecture
target.set_weights(model_action.get_weights())  # copy the weights

In [44]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [45]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [46]:
batch_size = 32
discount_factor = 0.5
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss_fn = tf.keras.losses.MeanSquaredError()


def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
        #################### CHANGED SECTION ####################
    next_Q_values = model_action.predict(next_states, verbose=0)  # ≠ target.predict()
    best_next_actions = next_Q_values.argmax(axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    max_next_Q_values = (target.predict(next_states, verbose=0) * next_mask
                        ).sum(axis=1)
    #########################################################
    next_Q_values = target.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [51]:
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=10000)


sumoCmd = [sumo_bin, "-c",simulConfig,'--start']
print(sumoCmd)
for episode in range(80):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(130000): ## TO CHANGED
        epsilon = max(1 - episode / 80, 0.01)

        if step%2000 == 0:
            #######################################################################
            next_state = np.array(get_state(lane_ids))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))
            #######################################################################

            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)

        traci.simulationStep()

    if episode%5==0:
        target.set_weights(model_action.get_weights())
    print(f'episode : {episode}')
    traci.close()


['/Users/arseneclaustre/sumo/bin/sumo', '-c', 'Traditional_traffic/traditional_traffic.sumo.cfg', '--start']
 Retrying in 1 seconds


episode : 0
Step #1300.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 924 ACT 52 BUF 0)                 
 Retrying in 1 seconds


episode : 1
Step #1300.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 924 ACT 56 BUF 0)                 
 Retrying in 1 seconds


state : [ 0  4 26  0  3  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 13 63  7  7 11  1  0  0  0  7  0  3  0  3  0  0  0  0  0  9  0  0  0]
Q_values : [-2.5035498 -5.188497  -7.715867   1.3268075]


state : [ 0 14 76  0 13 13  0  0  2  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0
  1 14 78  7 16 16  2  1  2  2  3  0  2  0  0  2  0  0  0  0  0  0  0  0]
Q_values : [ -2.0485034  -9.988745  -13.55791     9.19729  ]


...

state : [ 0  0  0  0 20 11  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  2  6 24 19  2  0  0  0  8  0  4  0  0  1  0  9  0  0  2  0  0  1]
Q_values : [ 3.3639197 10.003187  18.327744  13.253811 ]
state : [ 0  0  2  0  0 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  2  5 15 23  0  0  0  0  0  0  4  0  0  2  4  5  0  0  0  0  7  3]
Q_values : [-3.706398  -2.6080995  4.5540457  8.295361 ]


state : [ 0  2  0  0 10  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  2  0  0 14  0  0  2  0  0  0  0  0  0  1  0  0  0  0  0  9  0  1  0]
Q_values : [0.73946095 2.2742372  9.685255   5.170556  ]
state : [0 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 2 0 0 0 0 0
 0 0 0 0 0 0 0 3 0 7 3]
Q_values : [-8.392421  -7.650775   0.933589   4.8974667]


episode : 33
Step #1300.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 924 ACT 7 BUF 0)                  
 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ -9.006638  -9.942784 -10.338176 -11.356134]
state : [ 0  0 10  0  4  0  0  0  2  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1 13  4  5  0  4  0  2  1 13  0  2  0  1  4  0  0  0  0  3  0  0  0]
Q_values : [ 3.0380592 15.707311  15.395517   2.1938257]
state : [ 0  0  0  0  4  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  6  1  6  2  5  0  0  1 17  0  0  0  0  3  0 10  0  0  0  0  0  1]
Q_values : [-2.5450134  4.7492666 10.367809   0.6242216]
Step #100.00(0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 84 ACT 57 BUF 0)                    

state : [0 0 5 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9 2 3 3 4 0 0 0 8 0 2
 0 0 4 6 7 0 0 0 0 3 2]
Q_values : [-9.876295  -4.6068563 -2.6591592 -5.3889823]
state : [ 0  0  9  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1 12  0  0  4  4  0  0  1  2  0  2  0  0  4  5  3  0  0  0  0  1  0]
Q_values : [-2.098046   5.8091536  1.4217772  4.142748 ]
state : [0 0 0 0 0 4 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 5 2 3 5 5 3 1 3 7 1 0
 0 0 3 0 9 0 0 0 0 0 0]
Q_values : [ -9.642281 -10.309027 -10.441169  -9.524164]
state : [ 0  2  0  0  2  0  0  2  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  3  3  2  4  2  4  2  0  1 12  0  0  0  0  4  0  7  0  0  0  0  0  1]
Q_values : [-5.30495   -2.163301   4.9536743  1.1535667]
state : [0 2 3 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 2 0 2 5 2 0 1 4 0 2
 0 0 2 6 4 0 0 0 0 2 1]
Q_values : [-4.5821886  -0.7048907  -0.12347794  0.76654446]
Interrupt signal received, trying to exit gracefully.T 178 ACT 48 BUF 2)        

KeyboardInterrupt: 

In [ ]:
sumoCmd = [sumo_gui_bin, "-c",simulConfig,"--start", "--no-warnings"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(130000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        # action = np.random.randint(8)
        # print("action", action)
        # print(traci.trafficlight.getAllProgramLogics(nom_du_feu))
        # print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-5.0287285 -5.4861917 -5.2892523 -4.848423 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-10.795435  -11.943055  -12.3541155 -10.7902355]
state : [0 2 2 0 2 0 0 2 2 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 2 2 0 6 2 2 1 7 0 2
 0 2 2 0 0 2 0 2 2 0 0]
Q_values : [-6.6604815 -2.3230062  3.8427765 -1.3727925]
state : [ 0  2  6  0  0  0  0  2  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  2 10  2  1  2  4  2  2  0  3  0  0  0  0  4  4  2  0  0  0  0  2  0]
Q_values : [-3.7509315   2.073576    0.12481058  1.949066  ]
state : [0 2 0 0 1 2 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 3 4 2 2 4 2 0 1 6 0 2
 0 0 4 0 9 0 0 0 0 0 1]
Q_values : [-8.6424675 -5.243136  -1.0198042 -2.2205286]
state : [0 2 3 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 1 1 4 5 2 0 1 3 0 0
 0 0 3 4 4 0 0 0 0

FatalTraCIError: Connection closed by SUMO.